<h4><b>RNA Regression - Tuning de Parâmetros</b></h4>
<p>Neste experimento iremos realizar o Tuning de Parâmetros;</p>
<p>Utilizaremos a base de dados grande.</p>

In [2]:
#!pip install skorch

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from skorch import NeuralNetRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
torch.__version__

'2.0.1'

In [3]:
torch.manual_seed(123)

<h4><b>Carregando dados</b></h4>

In [4]:
path = '/media/marcos/500GB/00_datasets/post_processing/post_processing_dataset.csv'
df = pd.read_csv(path)

In [5]:
df = df[[
    'line_category',
    'line_color',
    'station_initial_latitude',
    'station_initial_longitude',
    'station_final_latitude',
    'station_final_longitude',
    'distance',
    'average_speed_kh',
    'duration'
]]

In [6]:
df

,line_category,line_color,station_initial_latitude,station_initial_longitude,station_final_latitude,station_final_longitude,distance,average_speed_kh,duration
0,1,1,-25.406891,-49.252828,-25.433741,-49.270292,5100.000000,20.400000,900.000000
1,8,5,-25.433741,-49.270292,-25.396167,-49.358341,17300.000000,38.444444,1620.000000
2,1,1,-25.396167,-49.358341,-25.435187,-49.271813,11300.000000,24.214286,1680.000000
3,2,2,-25.435187,-49.271813,-25.436659,-49.273807,1100.000000,16.500000,240.000000
4,2,2,-25.436659,-49.273807,-25.435052,-49.275876,2000.000000,15.000000,480.000000
...,...,...,...,...,...,...,...,...,...
12367,7,6,-25.424410,-49.295560,-25.494404,-49.282068,10500.000000,25.200000,1500.000000
12368,6,5,-25.494404,-49.282068,-25.475952,-49.240087,6200.000000,26.571429,840.000000
12369,7,6,-25.475952,-49.240087,-25.459501,-49.271815,5100.000000,25.500000,720.000000
12370,3,3,-25.459501,-49.271815,-25.440706,-49.271456,2500.000000,25.000000,360.000000


<h4><b>Atribuindo valores as variáveis X e y</b></h4>

In [7]:
X = df.iloc[:,0:8].values
y = df.iloc[:,8].values

In [8]:
X.shape

(12372, 8)

In [9]:
y.shape

(12372,)

<h4><b>Convertendo em tensores Pytorch</b></h4>
<p>view: Redefine as dimensões</p>

In [10]:
X = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float).view(-1,1)

In [11]:
type(X)

torch.Tensor

<h4><b>Confirmando se a variável y está em formato de matriz</b></h4>

In [12]:
y.shape

torch.Size([12372, 1])

<h4><b>Construção do modelo da rede neural</b></h4>
<p>Efetuamos este cálculo visando inicializar a quantidade de neurônios nas camadas ocultas da rede neural.</p>
<p>(entradas + saidas) / 2</p>

<ul>
    <li>Entrada: 8</li>
    <li>Camada oculta: 4 neurônios</li>
    <li>Camada oculta: 4 neurônios</li>
    <li>Camada de saída 1 neurônios</li>
</ul>

In [13]:
class regressor_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(8, 4)
        self.dense1 = nn.Linear(4, 4)
        self.dense2 = nn.Linear(4, 1)
        self.activation = nn.ReLU()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dense2(X)
        return X

<h4><b>Configurando treinamento</b></h4>

In [15]:
regressor_sklearn = NeuralNetRegressor(
    module=regressor_torch,
    optimizer=torch.optim.Adam,
    max_epochs=100,
    batch_size=64,
    train_split=False
)

<h4><b>Grid Search</b></h4>
<p>Acionando o GridSearchCV do Scikit-Learn para efetuar a busca da melhor combinação de hiperparâmetros;</p>
<p>Ele avaliará todas as combinações de valores possíveis por meio de validação cruzada;</p>
<p>Neste experimentos tentamos gerar através da busca do GridSearchCV a melhor combinações de valores para o hiperparâmetro criterion.</p>

In [16]:
params = {
    'criterion': [nn.MSELoss, nn.L1Loss, torch.nn.SmoothL1Loss],
    'optimizer__lr': [0.01, 0.001, 0.0001],
    'batch_size': [16, 32, 64],
    'max_epochs': [25, 50, 100],
}

In [17]:
grid_search = GridSearchCV(
    estimator=regressor_sklearn,
    param_grid=params,
    cv=5
)

In [18]:
grid_search = grid_search.fit(X, y)

  epoch    train_loss     dur
-------  ------------  ------
      1   132897.1666  0.7479
      2    93086.5413  0.7449
      3    79798.8233  0.7430
      4    76310.3385  0.7454
      5    73074.6003  0.7404
      6    69562.6917  0.7235
      7    65384.7630  0.7112
      8    59747.3448  0.7173
      9    51855.3570  0.7163
     10    41965.2186  0.7194
     11    34587.5259  0.7338
     12    30753.5509  0.7461
     13    29455.7084  0.7214
     14    28895.3767  0.7197
     15    28604.9248  0.7213
     16    28430.5270  0.7203
     17    28316.2974  0.7174
     18    28235.2140  0.7187
     19    28178.7349  0.7167
     20    28126.9603  0.7195
     21    28084.8955  0.7174
     22    28040.4887  0.7192
     23    28007.5847  0.7179
     24    27979.2223  0.7192
     25    27985.7849  0.7183
  epoch    train_loss     dur
-------  ------------  ------
      1   162499.5046  0.7193
      2    79291.8404  0.7183
      3    60213.7863  0.7180
      4    53892.9416  0.7160
      5   

In [19]:
best_params = grid_search.best_params_
best_params

{'batch_size': 64,
 'criterion': torch.nn.modules.loss.SmoothL1Loss,
 'max_epochs': 100,
 'optimizer__lr': 0.01}

In [22]:
best_result = grid_search.best_score_
best_result

0.9530152438633156